In [12]:
import pandas as pd
from spacy.lang.fr import French
import spacy
from line_profiler import LineProfiler
from time import time
from spacy.matcher import Matcher
from spacy.cli.download import download


In [13]:
download('fr_core_news_sm')


✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [2]:
output_filename= "../../data/processed/DPEFs/dpef_paragraphs.csv"
df_parsed_data = pd.read_csv(output_filename,sep=";")

In [3]:
df_parsed_data.loc[211]

SIREN                            552037806
project_denomination                 vinci
denomination                         VINCI
pdf_name                vinci_2018_ddr.pdf
page_nb                                210
paragraph_id                           104
paragraph                    55,7 % 24,6 %
x_group                                  0
y_min_paragraph                        472
y_max_paragraph                        492
Name: 211, dtype: object

In [16]:


def isPercent(x):

    doc = nlp(x)
    if len(doc)<10:
        return False
    # Iterate over the tokens in the doc
    for token in doc[:-1]:
        # Check if the token resembles a number
        if token.like_num:
            # Get the next token in the document
            next_token = doc[token.i+1]
            # Check if the next token's text equals '%'
            if next_token.text == "%":
                return True
            else:
                return False
        else:
            return False
        
        
def isFigure(x):

    doc = nlp(x)
    if len(doc)<10:
        return False
    # Iterate over the tokens in the doc
    for token in doc[:-1]:
        # Check if the token resembles a number
        if token.like_num:
            return True
        else:
            return False
        
"""https://course.spacy.io/chapter1"""
def isCommit(x):
 
    doc = nlp(x)

    matches = matcher(doc)
    #print("Total matches found:", len(matches))
    if matches:
        return True
    else:
        return False
    # Iterate over the matches and print the span text
    #for match_id, start, end in matches:
        #print("Match found:", doc[start:end].text);

In [5]:
#nlp = French()
#df_parsed_data['Is_commit']=df_parsed_data['paragraph'].apply(isCommit)

In [6]:
nlp = French()
df_parsed_data['Is_cfigure']=df_parsed_data['paragraph'].apply(isFigure)

In [15]:
nlp = spacy.load("fr_core_news_sm")
matcher = Matcher(nlp.vocab)
# Write a pattern that matches a form of "download" plus proper noun
pattern = [{"LEMMA": "engager"}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("ENGAGEMENT", None, pattern)

In [17]:
df_parsed_data['is_commit']=df_parsed_data['paragraph'].apply(isCommit)

In [20]:
df_parsed_data[df_parsed_data['is_commit']==True]['paragraph'].values


array(['E N G A G E M E N T D U M A N I F E S T E V I N C I « Nous refusons de considérer les accidents du travail comme une fatalité, et nous nous engageons sur l’objectif du “zéro accident”. »',
       'E N G A G E M E N T D U M A N I F E S T E V I N C I « Nous nous engageons à réduire nos émissions de gaz à effet de serre de 30 % à l’horizon 2020, à accompagner nos clients dans la recherche d’une meilleure efficacité énergétique et à les inciter à adopter un comportement écoresponsable. »',
       'Les entreprises de VINCI élaborent des solutions pour aider leurs clients à réduire leurs propres impacts environnementaux. Oxygen®2, offre d’écoconception développée par VINCI Construction France et VINCI Facilities (VINCI Energies), modélise un bâtiment énergétiquement efficace tout au long de son cycle de vie, de la conception à l’exploitation. VINCI Construction France a participé activement, en 2018, à l’expérimentation du référentiel E+C– (bâtiment à énergie positive et réduction ca